In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time
import re

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [2]:
DATABASE_NAME = "internet_governance_news.db"

def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela 'articles' prontos!")

create_database()


✅ Banco e tabela 'articles' prontos!


In [3]:
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


In [4]:
def montar_url(pagina):
    if pagina == 1:
        return "https://www12.senado.leg.br/noticias/ultimas"
    return f"https://www12.senado.leg.br/noticias/ultimas/{pagina}"


In [5]:
noticias = []

for pagina in range(18, 0, -1):
    url = montar_url(pagina)
    print(f"📄 Coletando página {pagina}: {url}")

    r = requests.get(url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    lista = soup.find("ol", class_="lista-resultados")
    if not lista:
        print("⚠️ Nenhuma lista encontrada")
        continue

    itens = lista.find_all("li")
    print(f"   {len(itens)} notícias encontradas")

    for item in itens:
        titulo_tag = item.find("span", class_="eta")
        link_tag = item.find("a")
        data_tag = item.select_one("div.text-muted.normalis")

        if not titulo_tag or not link_tag:
            continue

        data = {
            "title": titulo_tag.get_text(strip=True),
            "date": data_tag.get_text(strip=True) if data_tag else None,
            "author": "Agência Senado",
            "url": "https://www12.senado.leg.br" + link_tag["href"],
            "source": "Senado Federal"
        }

        noticias.append(data)
        insert_article(**data)

    time.sleep(1)

print(f"\n✅ Total coletado: {len(noticias)} notícias")
df_senado = pd.DataFrame(noticias)
display(df_senado.head())


📄 Coletando página 18: https://www12.senado.leg.br/noticias/ultimas/18
   20 notícias encontradas
📄 Coletando página 17: https://www12.senado.leg.br/noticias/ultimas/17
   20 notícias encontradas
📄 Coletando página 16: https://www12.senado.leg.br/noticias/ultimas/16
   20 notícias encontradas
📄 Coletando página 15: https://www12.senado.leg.br/noticias/ultimas/15
   20 notícias encontradas
📄 Coletando página 14: https://www12.senado.leg.br/noticias/ultimas/14
   20 notícias encontradas
📄 Coletando página 13: https://www12.senado.leg.br/noticias/ultimas/13
   20 notícias encontradas
📄 Coletando página 12: https://www12.senado.leg.br/noticias/ultimas/12
   20 notícias encontradas
📄 Coletando página 11: https://www12.senado.leg.br/noticias/ultimas/11
   20 notícias encontradas
📄 Coletando página 10: https://www12.senado.leg.br/noticias/ultimas/10
   20 notícias encontradas
📄 Coletando página 9: https://www12.senado.leg.br/noticias/ultimas/9
   20 notícias encontradas
📄 Coletando página 8: 

,title,date,author,url,source
0,Avança proposta de criação de exame nacional o...,03/12/2025 13h21,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
1,Cleitinho defende Michelle em polêmica sobre a...,03/12/2025 12h28,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
2,CCJ aprova regras para testamento de emergência,03/12/2025 12h12,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
3,Agora é lei: doação de medicamentos será isent...,03/12/2025 11h37,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
4,Vai ao Plenário PEC que amplia acúmulo de carg...,03/12/2025 10h55,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal


In [6]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT * FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total no banco: {len(df_db)} registros")
display(df_db.head(20))


📦 Total no banco: 2573 registros


,id,title,date,author,url,source
0,683,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,1374,CGI.br busca saídas para o acesso mundial à re...,31 OUT 2007,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-busca-s...,CGI Notícias
2,1399,ICANN 2006 debate no Brasil as tendências da i...,31 OUT 2006,CGI.br Notícias,https://cgi.br/noticia/releases/icann-2006-deb...,CGI Notícias
3,797,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
4,1140,TIC Empresas 2013,31 MAI 2014,CGI.br Notícias,https://cgi.br/noticia/notas/tic-empresas-2013/,CGI Notícias
5,1267,CGI.br apresenta TIC Domicílios 2011,31 MAI 2012,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-apresen...,CGI Notícias
6,1411,Comitê Gestor da Internet no Brasil comemora d...,31 MAI 2005,CGI.br Notícias,https://cgi.br/noticia/releases/comite-gestor-...,CGI Notícias
7,1412,Comitê Gestor da Internet no Brasil comemo...,31 MAI 2005,CGI.br Notícias,https://cgi.br/noticia/coletivas/comite-gestor...,CGI Notícias
8,1428,O terceiro milênio - Passado e futuro na virad...,31 DEZ 2000,CGI.br Notícias,https://cgi.br/noticia/releases/o-terceiro-mil...,CGI Notícias
9,880,Dificuldade dos pais para apoiar alunos e falt...,31 AGO 2021,CGI.br Notícias,https://cgi.br/noticia/releases/dificuldade-do...,CGI Notícias


In [7]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para plotar")
        return

    # Top 15
    top15 = df.head(15).copy()
    top15["rank"] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x="rank",
        y="title",
        orientation="h",
        title="Top 15 Notícias – Internet Governance"
    )
    fig1.update_layout(height=600)
    fig1.show()

    # Fonte
    source_count = df["source"].value_counts().reset_index()
    source_count.columns = ["source", "count"]

    fig2 = px.pie(
        source_count,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()

    # Palavras
    text = " ".join(df["title"].astype(str)).lower()
    words = re.findall(r"\b\w{4,}\b", text)

    word_freq = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    word_freq.columns = ["palavra", "freq"]

    fig3 = px.treemap(
        word_freq,
        path=["palavra"],
        values="freq",
        title="Palavras mais frequentes nos títulos"
    )
    fig3.show()


In [8]:
plot_charts(df_db)